In [22]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
from scipy.stats import shapiro, f_oneway, kruskal, chi2_contingency
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline

In [23]:
df = pd.read_csv('StudentsPerformance.csv')
df['average score'] = df[['math score', 'reading score', 'writing score']].mean(axis=1)

In [24]:
print("1. Проверка нормальности распределений в группах по уровню образования родителей\n")

education_groups = {}
normal_groups = []
non_normal_groups = []
group_names = df['parental level of education'].unique()

for name in group_names:
    group = df[df['parental level of education'] == name]['average score'].dropna()
    education_groups[name] = group

    stat, p_val = shapiro(group)
    print(f"Shapiro-Wilk для группы '{name}': p-value = {p_val:.3f}")

    if p_val < 0.05:
        print(f"❌ Распределение НЕ нормальное для группы '{name}'")
        non_normal_groups.append(name)
    else:
        print(f"✅ Распределение нормальное для группы '{name}'")
        normal_groups.append(name)
    print()

1. Проверка нормальности распределений в группах по уровню образования родителей

Shapiro-Wilk для группы 'bachelor's degree': p-value = 0.340
✅ Распределение нормальное для группы 'bachelor's degree'

Shapiro-Wilk для группы 'some college': p-value = 0.051
✅ Распределение нормальное для группы 'some college'

Shapiro-Wilk для группы 'master's degree': p-value = 0.230
✅ Распределение нормальное для группы 'master's degree'

Shapiro-Wilk для группы 'associate's degree': p-value = 0.110
✅ Распределение нормальное для группы 'associate's degree'

Shapiro-Wilk для группы 'high school': p-value = 0.105
✅ Распределение нормальное для группы 'high school'

Shapiro-Wilk для группы 'some high school': p-value = 0.009
❌ Распределение НЕ нормальное для группы 'some high school'



In [25]:
print("\n2. ANOVA для групп с нормальным распределением\n")

if len(normal_groups) > 1:
    normal_data = [education_groups[name] for name in normal_groups]
    f_stat, p_val = f_oneway(*normal_data)
    print(f"ANOVA: F-statistic = {f_stat:.3f}, P-value = {p_val:.3f}")

    if p_val < 0.05:
        print("✅ Отклоняем H₀: существуют статистически значимые различия между группами.")
    else:
        print("❌ Не отклоняем H₀: статистически значимых различий между группами нет.")
else:
    print("Недостаточно групп с нормальным распределением для ANOVA.")


2. ANOVA для групп с нормальным распределением

ANOVA: F-statistic = 11.857, P-value = 0.000
✅ Отклоняем H₀: существуют статистически значимые различия между группами.


In [26]:
print("\n3. Хи-квадрат тест после категоризации данных\n")
df['score_category'] = pd.qcut(df['average score'], q=3, labels=['низкий', 'средний', 'высокий'])

contingency_table = pd.crosstab(df['parental level of education'], df['score_category'])
print("Таблица сопряженности (образование родителей × категория балла):")
print(contingency_table)
print()


3. Хи-квадрат тест после категоризации данных

Таблица сопряженности (образование родителей × категория балла):
score_category               низкий  средний  высокий
parental level of education                          
associate's degree               67       68       87
bachelor's degree                28       42       48
high school                      83       73       40
master's degree                  14       16       29
some college                     72       79       75
some high school                 72       54       53



In [27]:
chi2, p, dof, expected = chi2_contingency(contingency_table)
print(f"Хи-квадрат: χ² = {chi2:.3f}, df = {dof}, p-value = {p:.5f}")

if p < 0.05:
    print(
        "✅ Отклоняем H₀: есть статистически значимая связь между уровнем образования родителей и категорией среднего балла.")
else:
    print(
        "❌ Не отклоняем H₀: статистически значимой связи между уровнем образования родителей и категорией среднего балла нет.")

Хи-квадрат: χ² = 35.148, df = 10, p-value = 0.00012
✅ Отклоняем H₀: есть статистически значимая связь между уровнем образования родителей и категорией среднего балла.


In [30]:
# Дополнительная проверка зависимости между подготовкой к тесту и образованием родителей
print("\n5. Проверка зависимости между курсом подготовки и образованием родителей\n")

prep_table = pd.crosstab(df['parental level of education'], df['test preparation course'])
print("Таблица сопряженности (образование родителей × подготовка к тесту):")
print(prep_table)
print()


5. Проверка зависимости между курсом подготовки и образованием родителей

Таблица сопряженности (образование родителей × подготовка к тесту):
test preparation course      completed  none
parental level of education                 
associate's degree                  82   140
bachelor's degree                   46    72
high school                         56   140
master's degree                     20    39
some college                        77   149
some high school                    77   102



In [31]:
chi2_prep, p_prep, dof_prep, expected_prep = chi2_contingency(prep_table)
print(f"Хи-квадрат: χ² = {chi2_prep:.3f}, df = {dof_prep}, p-value = {p_prep:.5f}")

if p_prep < 0.05:
    print(
        "✅ Отклоняем H₀: есть статистически значимая связь между уровнем образования родителей и прохождением курса подготовки.")
else:
    print(
        "❌ Не отклоняем H₀: статистически значимой связи между уровнем образования родителей и прохождением курса подготовки нет.")

Хи-квадрат: χ² = 9.544, df = 5, p-value = 0.08923
❌ Не отклоняем H₀: статистически значимой связи между уровнем образования родителей и прохождением курса подготовки нет.


In [32]:
#  Влияние образования родителей на средний балл (регрессионный анализ)
print("\n6. Влияние образования родителей на средний балл (регрессионный анализ)\n")

# Преобразование категориальных переменных
categorical_features = ['gender', 'race/ethnicity', 'parental level of education', 'lunch', 'test preparation course']
X = df[categorical_features]
y = df['average score']


6. Влияние образования родителей на средний балл (регрессионный анализ)



In [33]:
# One-hot encoding для категориальных переменных
preprocessor = ColumnTransformer(
    transformers=[
        ('cat', OneHotEncoder(), categorical_features)
    ])

In [34]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

pipe = Pipeline([
    ('preprocessor', preprocessor),
    ('regressor', LinearRegression())
])

pipe.fit(X_train, y_train)
y_pred = pipe.predict(X_test)

mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

print(f"Mean Squared Error: {mse:.3f}")
print(f"R² score: {r2:.3f}")

Mean Squared Error: 176.157
R² score: 0.219


In [37]:

feature_names = pipe.named_steps['preprocessor'].transformers_[0][1].get_feature_names_out(categorical_features)
education_features = [f for f in feature_names if 'parental level of education' in f]
education_coefs = pipe.named_steps['regressor'].coef_[
    [i for i, name in enumerate(feature_names) if name in education_features]
]

print("\nВлияние уровня образования родителей на средний балл:")
for feature, coef in zip(education_features, education_coefs):
    level = feature.replace('parental level of education_', '')
    print(f"{level}: {coef:.3f}")


Влияние уровня образования родителей на средний балл:
associate's degree: 0.578
bachelor's degree: 4.016
high school: -4.176
master's degree: 1.726
some college: 0.281
some high school: -2.426


In [36]:
print("\n\nВЫВОДЫ:")
print("1. Тест Шапиро-Уилка показал, что в группе 'some high school' распределение не является нормальным,")
print("   что указывает на необходимость использования непараметрических методов для этой группы.")
print("2. ANOVA для групп с нормальным распределением выявила статистически значимые различия средних баллов.")
print("3. Хи-квадрат тест после категоризации баллов подтвердил наличие статистически значимой связи между")
print("   уровнем образования родителей и уровнем успеваемости студентов.")
print("4. Связи между прохождением курса подготовки и уровнем образования родителей не обнаружено.")
print("5. Регрессионный анализ выявил влияние образования родителей на успеваемость, где наибольший положительный")
print("   эффект наблюдается у родителей с высшим образованием (магистр, бакалавр).")



ВЫВОДЫ:
1. Тест Шапиро-Уилка показал, что в группе 'some high school' распределение не является нормальным,
   что указывает на необходимость использования непараметрических методов для этой группы.
2. ANOVA для групп с нормальным распределением выявила статистически значимые различия средних баллов.
3. Хи-квадрат тест после категоризации баллов подтвердил наличие статистически значимой связи между
   уровнем образования родителей и уровнем успеваемости студентов.
4. Связи между прохождением курса подготовки и уровнем образования родителей не обнаружено.
5. Регрессионный анализ выявил влияние образования родителей на успеваемость, где наибольший положительный
   эффект наблюдается у родителей с высшим образованием (магистр, бакалавр).
